This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
stock_exchange = 'FSE'
stock_ticker = 'AFX_X'
return_format = 'json'
start_date = '2020-06-15'
end_date = '2020-06-16'
parameters = {'start_date':start_date, 'end_date':end_date}

url = f"https://data.nasdaq.com/api/v3/datasets/{stock_exchange}/{stock_ticker}/data.{return_format}"

res = requests.get(url, parameters)
print(res.status_code)

200


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response = res.json()
print(response)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-06-15', 'end_date': '2020-06-16', 'frequency': 'daily', 'data': [['2020-06-16', 88.35, 90.75, 87.45, 90.65, None, 206243.0, 18478591.35, None, None, None], ['2020-06-15', 85.0, 86.9, 84.75, 86.35, None, 181008.0, 15502164.95, None, None, None]], 'collapse': None, 'order': None}}


## These are your tasks for this mini project:

1. [Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).](#1)
2. [Convert the returned JSON object into a Python dictionary.](#2)
3. [Calculate what the highest and lowest opening prices were for the stock in this period.](#3)
4. [What was the largest change in any one day (based on High and Low price)?](#4)
5. [What was the largest change between any two days (based on Closing Price)?](#5)
6. [What was the average daily trading volume during this year?](#6)
7. [(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)](#7)

<a name = '1'></a>


<em>Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).</em>

In [5]:
stock_exchange = 'FSE'
stock_ticker = 'AFX_X'
return_format = 'json'
start_date = '2017-01-01'
end_date = '2018-01-01'
parameters = {'start_date':start_date, 'end_date':end_date}

url = f"https://data.nasdaq.com/api/v3/datasets/{stock_exchange}/{stock_ticker}/data.{return_format}"

res = requests.get(url, parameters)
print(res.status_code)

200


<a name = '2'></a>

<em>Convert the returned JSON object into a Python dictionary.</em>

In [6]:
json_file = res.json()
print(json_file['dataset_data'].keys())

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


<a name = '3'></a>

<em>Calculate what the highest and lowest opening prices were for the stock in this period.</em>

In [7]:
# Get the index of open price in data lists:
print(json_file['dataset_data']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [8]:
highest_op = json_file['dataset_data']['data'][0][1]
lowest_op = json_file['dataset_data']['data'][0][1]

for day in json_file['dataset_data']['data']:
    if day[1]:
        highest_op = max(highest_op, day[1])
        lowest_op = min(lowest_op, day[1])

print(f'The highest opening price in 2017 was ${highest_op}')
print(f'The lowest opening price in 2017 was ${lowest_op}')

The highest opening price in 2017 was $53.11
The lowest opening price in 2017 was $34.0


<a name = '4'></a>

<em>What was the largest change in any one day (based on High and Low price)?</em>

In [9]:
largest_change = 0

for day in json_file['dataset_data']['data']:
    change = abs(day[2] - day[3])
    if change:
        largest_change = max(largest_change, change)


print(f'The largest change in any one day in 2017 was ${round(largest_change,2)}')

The largest change in any one day in 2017 was $2.81


<a name = '5'></a>

<em>What was the largest change between any two days (based on Closing Price)?</em>

In [10]:
largest_2day_change = 0
last_day_close = json_file['dataset_data']['data'][0][4]

for day in json_file['dataset_data']['data']:
    change = abs(day[4] - last_day_close)
    largest_2day_change = max(largest_2day_change, change)    
    last_day_close = day[4]

print(f'The largest change between any two days is ${round(largest_2day_change,2)}')

The largest change between any two days is $2.56


<a name = '6'></a>

<em>What was the average daily trading volume during this year?</em>

In [11]:
s = 0

for day in json_file['dataset_data']['data']:
    s += day[6]

av_tr_vol = round(s/len(json_file['dataset_data']['data']),2)
print(f'Average daily trading volume during 2017 was ${av_tr_vol}')

Average daily trading volume during 2017 was $89124.34


<a name = '7'></a>

<em>(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)</em>

In [12]:
tr_vol = []
for day in json_file['dataset_data']['data']:
    tr_vol.append(day[6])
tr_vol.sort()
ind = int(len(tr_vol)/2)
print(f'The median trading volume during this year was ${tr_vol[ind]}')

The median trading volume during this year was $76286.0
